In [4]:
import os
import sys
import time
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Callable

import einops
import numpy as np
import torch as t
import torch.nn as nn
import wandb
from eindex import eindex
from jaxtyping import Float, Int
from rich import print as rprint
from rich.table import Table
from tabulate import tabulate
from torch import Tensor
from transformer_lens import HookedTransformer, utils
from transformer_lens.hook_points import HookPoint


device = t.device("mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu")
BASE_MODEL = "gpt2-small"

In [5]:

@dataclass
class RLHFArgs:
    # Basic / global
    seed: int = 1

    # Wandb / logging
    use_wandb: bool = False
    wandb_project_name: str = "RLHF"
    wandb_entity: str | None = None

    # Duration of different phases
    total_phases: int = 100
    batch_size: int = 128
    num_minibatches: int = 4
    batches_per_learning_phase: int = 2

    # Optimization hyperparameters
    base_lr: float = 2e-5
    head_lr: float = 5e-4
    max_grad_norm: float = 1.0
    warmup_steps: int = 20
    final_scale: float = 0.1

    # Computing other PPO loss functions
    clip_coef: float = 0.2
    vf_coef: float = 0.15
    ent_coef: float = 0.001

    # Base model & sampling arguments
    base_model: str = BASE_MODEL
    gen_len: int = 30
    temperature: float = 1.0
    top_k: int = 10
    prefix: str = "This is"
    prepend_bos: bool = True

    # RLHF-specific arguments
    kl_coef: float = 2.5
    reward_fn: Callable = lambda x: 0.0
    normalize_reward: bool = True

    def __post_init__(self):
        assert self.total_phases > self.warmup_steps, "total_phases must be greater than warmup_steps"
        assert self.batch_size % self.num_minibatches == 0, "batch_size should be divisible by num_minibatches"
        self.minibatch_size = self.batch_size // self.num_minibatches

In [6]:

class TransformerWithValueHead(nn.Module):
    """
    Defines a GPT model with a value head (the latter taking the last hidden state as input, post-layernorm).

    The value head is a simple MLP with one hidden layer, and scalar output:

        Linear(d_model -> 4*d_model)
        ReLU
        Linear(4*d_model -> 1)

    All linear layers have biases.
    """

    base_model: HookedTransformer
    value_head: nn.Sequential

    def __init__(self, base_model):
        super().__init__()
        self.base_model = HookedTransformer.from_pretrained(base_model)

        d_model = self.base_model.cfg.d_model
        self.value_head = nn.Sequential(nn.Linear(d_model, 4 * d_model), nn.ReLU(), nn.Linear(4 * d_model, 1))

    def forward(
        self, input_ids: Int[Tensor, "batch seq"]
    ) -> tuple[Float[Tensor, "batch seq d_vocab"], Int[Tensor, "batch seq"]]:
        value_head_output = None

        def calc_and_store_value_head_output(resid_post: Float[Tensor, "batch seq d_model"], hook: HookPoint):
            nonlocal value_head_output
            value_head_output = self.value_head(resid_post).squeeze(-1)

        logits = self.base_model.run_with_hooks(
            input_ids,
            return_type="logits",
            fwd_hooks=[(utils.get_act_name("normalized"), calc_and_store_value_head_output)],
        )
        return logits, value_head_output


# Define a reference model (we'll use this during RLHF)
model = TransformerWithValueHead(BASE_MODEL).to(device)

# Test your value head's architecture
assert isinstance(model.base_model, HookedTransformer)
assert isinstance(model.value_head, nn.Module)
d_model = model.base_model.cfg.d_model
n_params_expected = (d_model + 1) * 4 * d_model + (4 * d_model + 1)
assert len(model.value_head) == 3, "Your value head should be a `nn.Sequential` with 3 layers."
assert sum(p.numel() for p in model.value_head.parameters()) == n_params_expected, "Unexpected param count"

# Test your class's forward pass
batch_size, seq_len = 2, 10
input_ids = t.randint(0, 1000, (batch_size, seq_len)).to(device)
logits, values = model(input_ids)
assert logits.shape == (batch_size, seq_len, model.base_model.cfg.d_vocab), "logits should be (batch, seq, d_vocab)"
assert values.shape == (batch_size, seq_len), "value head output should be (batch, seq)"

Loaded pretrained model gpt2-small into HookedTransformer


In [7]:

@t.no_grad()
def get_samples(
    base_model: HookedTransformer,
    prompt: str,
    batch_size: int,
    gen_len: int,
    temperature: float,
    top_k: int,
    prepend_bos: bool,
) -> tuple[Int[Tensor, "batch seq"], list[str]]:
    """
    Generates samples from the model, which will be fed into the reward model and evaluated.

    Inputs:
        base_model: the transformer to generate samples from (we don't need the value head)
        prompt: the initial prompt fed into the model
        batch_size: the number of samples to generate
        gen_len: the length of the generated samples (i.e. the number of *new* tokens to generate)
        temperature: the temperature of the sampling distribution (higher means more random completions)
        top_k: the topk parameter of sampling (higher means a wider variety of possible completions)

    Returns:
        sample_ids: the token ids of the generated samples (including initial prompt)
        samples: the generated samples (including initial prompt)
    """
    # Make sure we've passed in the base model (the bit we use for sampling)
    assert not isinstance(base_model, TransformerWithValueHead), "Please pass in the base model, not the model wrapper."

    # Convert our prompt into tokens
    input_ids = base_model.to_tokens(prompt, prepend_bos=prepend_bos).squeeze(0)

    # Generate samples
    output_ids = base_model.generate(
        input_ids.repeat(batch_size, 1),  # repeating single sequence along batch dim
        max_new_tokens=gen_len,
        stop_at_eos=False,
        temperature=temperature,
        top_k=top_k,
        verbose=False,
    )
    samples = base_model.to_string(output_ids)

    return output_ids.clone(), samples

In [8]:
sample_ids, samples = get_samples(
    model.base_model,
    prompt="So long, and thanks for all the",
    batch_size=5,
    gen_len=15,
    temperature=0.8,
    top_k=15,
    prepend_bos=False,
)

table = Table("Token IDs", "Samples", title="Demo of `sample` function", show_lines=True)
for ids, sample in zip(sample_ids, samples):
    table.add_row(str(ids.tolist()), repr(sample))

rprint(table)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


                                             Demo of `sample` function                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Token IDs                                              ┃ Samples                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1842, 11,    │ 'So long, and thanks for all the                       │
│ 198, 198, 42, 6996, 198, 198, 47, 5493, 2988, 13, 198, │ love,\n\nKerry\n\nProud father.\n\nA'                  │
│ 198, 32]                                               │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1104, 13,    │ 'So long, and thanks for all the                       │
│ 198, 198, 12, 50, 312, 198, 198, 14592, 50256, 818,    │ support.\n\n-Sid\n\nAdvertisements<|endoftext|>In the  │
│ 262, 14324, 11]                                        │ meantime,'                                             │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 7932, 661,   │ 'So long, and thanks for all the wonderful people who  │
│ 508, 423, 8639, 523, 42088, 11, 290, 5176, 284, 2506,  │ have contributed so generously, and thanks to everyone │
│ 508, 468, 925]                                         │ who has made'                                          │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 922, 1705,   │ "So long, and thanks for all the good news.\n\nI've    │
│ 13, 198, 198, 40, 1053, 550, 867, 7007, 287, 262,      │ had many requests in the past about the"               │
│ 1613, 546, 262]                                        │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ [2396, 890, 11, 290, 5176, 329, 477, 262, 1049, 670,   │ 'So long, and thanks for all the great work. Thanks    │
│ 13, 6930, 329, 3555, 0, 198, 198, 14592, 50256, 50,    │ for reading!\n\nAdvertisements<|endoftext|>SciFi is'   │
│ 979, 10547, 318]                                       │                                                        │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

In [9]:

def reward_fn_char_count(generated_sample: list[str], char: str = ".") -> Float[Tensor, "batch"]:
    """
    Reward function (counting number of instances of a particular character), evaluated on the generated samples. The
    return type should be a tensor of floats.
    """
    return t.tensor([item.count(char) for item in generated_sample], device=device, dtype=t.float)


# Test your reward function
A = "This is a test."
B = "......"
C = "Whatever"

t.testing.assert_close(reward_fn_char_count([A]), t.tensor([1.0], device=device))
t.testing.assert_close(reward_fn_char_count([A, B, C]), t.tensor([1.0, 6.0, 0.0], device=device))
t.testing.assert_close(reward_fn_char_count([A], " "), t.tensor([3.0], device=device))
print("All tests for `reward_fn_char_count` passed!")

All tests for `reward_fn_char_count` passed!


In [10]:
def normalize_reward(reward: Float[Tensor, "batch"], eps=1e-5) -> Float[Tensor, "batch"]:
    """
    Normalizes the reward function values over the batch of sequences.
    """
    return (reward - reward.mean()) / (reward.std() + eps)


# Test your reward normalization function
reward = 10 + 5 * t.randn(10_000)
reward_normalized = normalize_reward(reward)
assert reward_normalized.mean().abs() < 1e-4
assert (reward_normalized.std() - 1).abs() < 1e-4
# Test edge case of zero reward
reward = t.zeros(5)
reward_normalized = normalize_reward(reward)
assert reward_normalized.abs().sum() < 1e-4

print("All tests for `normalize_reward` passed!")

All tests for `normalize_reward` passed!


In [12]:
@t.no_grad()
def compute_advantages(
    values: Float[Tensor, "minibatch_size seq_len"],
    rewards: Float[Tensor, "minibatch_size"],
    prefix_len: int,
) -> Float[Tensor, "minibatch_size gen_len"]:
    """
    Computes the advantages for the PPO loss function, i.e. A_pi(s, a) = Q_pi(s, a) - V_pi(s).

    In this formula we replace Q(s, a) with the 1-step Q estimates, and V(s) with the 0-step value estimates.

    Inputs:
        values:
            the value estimates for each token in the generated sequence
        rewards:
            the rewards for the entire generated sequence
        prefix_len:
            the length of the prefix (i.e. the length of the initial prompt)

    Returns:
        advantages:
            the advantages for each token in the generated sequence (not the entire sequence)
    """
    # (see diagram) stack values [3, 4, 5, 6] and rewards [7,] to get the first term in our calculation of advantages
    one_step_q_est = t.cat([values[:, prefix_len:-1], rewards[:, None]], dim=-1)

    # (see diagram) slice values [2, 3, 4, 5, 6] to get our zero-step value estimates
    zero_step_value_est = values[:, prefix_len - 1 : -1]

    advantages = one_step_q_est - zero_step_value_est
    return advantages


In [13]:

@dataclass
class ReplayMinibatch:
    """
    Samples from the replay memory.
    """

    sample_ids: Float[Tensor, "minibatch_size seq_len"]
    logprobs: Float[Tensor, "minibatch_size gen_len"]
    advantages: Float[Tensor, "minibatch_size gen_len"]
    returns: Float[Tensor, "minibatch_size gen_len"]
    ref_logits: Float[Tensor, "minibatch_size seq_len d_vocab"]


class ReplayMemory:
    def __init__(
        self,
        args: RLHFArgs,
        sample_ids: Float[Tensor, "batch_size seq_len"],
        logprobs: Float[Tensor, "batch_size gen_len"],
        advantages: Float[Tensor, "batch_size gen_len"],
        values: Float[Tensor, "batch_size seq_len"],
        ref_logits: Float[Tensor, "batch_size seq_len d_vocab"],
    ):
        """
        Initializes the replay memory, with all the data generated from the rollout phase at once.

        The advantages are (batch_size, gen_len) because we only compute advantages for the generated
        tokens. The other tensors, except logprobs, uses seq_len instead of gen_len because they are
        computed for all tokens.
        """

        assert ref_logits.ndim == 3
        assert ref_logits.shape[0] == args.batch_size
        assert sample_ids.shape == values.shape == ref_logits.shape[:2]
        assert advantages.shape == logprobs.shape == (args.batch_size, args.gen_len)

        self.args = args
        self.sample_ids = sample_ids
        self.logprobs = logprobs
        self.advantages = advantages
        self.values = values
        self.ref_logits = ref_logits

    def get_minibatches(self) -> list[ReplayMinibatch]:
        """
        Generates a list of minibatches by randomly sampling from the replay memory. Each sequence appears
        exactly `batches_per_learning_phase` times in total.
        """
        minibatches = []

        returns = self.advantages + self.values[:, -self.args.gen_len - 1 : -1]

        for _ in range(self.args.batches_per_learning_phase):
            for indices in t.randperm(self.args.batch_size).reshape(self.args.num_minibatches, -1):
                minibatches.append(
                    ReplayMinibatch(
                        sample_ids=self.sample_ids[indices],
                        logprobs=self.logprobs[indices],
                        advantages=self.advantages[indices],
                        returns=returns[indices],
                        ref_logits=self.ref_logits[indices],
                    )
                )

        return minibatches

In [15]:

def calc_kl_penalty(
    logits: Float[Tensor, "minibatch_size gen_len d_vocab"],
    ref_logits: Float[Tensor, "minibatch_size gen_len d_vocab"],
    kl_coef: float,
    gen_len: int,
) -> Float[Tensor, ""]:
    """
    Computes the KL divergence between the logits and the reference logits, scaled
    by the penalty function. This is used to stop the learned policy from diverging
    too much from the original reference model's policy.

    logits:
        The logits for all generated tokens (under the new model).
    ref_logits:
        The logits for the generated tokens (under the reference model).
    kl_coef:
        The coefficient of the KL penalty.
    prefix_len:
        The length of the prefix to ignore when computing the KL divergence.
    """
    assert (
        logits.shape[1] == ref_logits.shape[1] == gen_len
    ), "Should pass in logits and ref_logits for all generated tokens only, i.e. [:, -gen_len-1: -1]"

    ref_logprobs = ref_logits.log_softmax(-1)
    logprobs = logits.log_softmax(-1)
    probs = logprobs.exp()

    kl_div = (probs * (logprobs - ref_logprobs)).sum(-1)

    return kl_coef * kl_div.mean()

In [16]:
def calc_entropy_bonus(
    logits: Float[Tensor, "minibatch_size gen_len d_vocab"], ent_coef: float, gen_len: int
) -> Float[Tensor, ""]:
    """
    Return the entropy bonus term, suitable for gradient ascent.

    logits:
        the logits of the tokens generated by the model before each generated token
    ent_coef:
        the coefficient for the entropy loss, which weights its contribution to the overall objective function.
    prefix_len:
        The length of the prefix to ignore when computing the KL divergence.
    """
    assert logits.shape[1] == gen_len, "Should pass in logits _before_ all generated tokens, i.e. [:, -gen_len-1: -1]"

    logprobs = logits.log_softmax(dim=-1)
    probs = logprobs.exp()
    entropy = -(probs * logprobs).sum(dim=-1)
    return ent_coef * entropy.mean()

In [17]:

def calc_value_function_loss(
    values: Float[Tensor, "minibatch_size gen_len"],
    mb_returns: Float[Tensor, "minibatch_size gen_len"],
    vf_coef: float,
    gen_len: int,
) -> Float[Tensor, ""]:
    """Compute the value function portion of the loss function.

    Note that for RLHF with advantages = TD residuals rather than GAE, this is equivalent to penalizing the squared
    error between values[t] and mb_values[t+1]. This is essentially equivalent to our TD loss expression for DQN, where
    we penalized the current network's Q values and the next-step target network Q values. The role is the same in
    both cases: to improve the accuracy (and reduce the variance) of our value function estimates.

    values:
        the value function predictions for the sampled minibatch, for all generated tokens (using the updated critic
        network)
    mb_returns:
        the target for our updated critic network (computed as `advantages + values` from the old network)
    vf_coef:
        the coefficient for the value loss, which weights its contribution to the overall loss. Denoted by c_1 in the paper.
    gen_len:
        the number of generated tokens, used for shape checking
    """
    assert values.shape[1] == gen_len, "Should pass in values before all generated tokens, i.e. [:, -gen_len-1: -1]"
    assert mb_returns.shape[1] == gen_len, "Should pass in returns before all generated tokens only"

    return 0.5 * vf_coef * (values - mb_returns).pow(2).mean()


def calc_clipped_surrogate_objective(
    logprobs: Float[Tensor, "minibatch_size gen_len"],
    mb_logprobs: Float[Tensor, "minibatch_size gen_len"],
    mb_advantages: Float[Tensor, "minibatch_size gen_len"],
    clip_coef: float,
    gen_len: int,
    eps: float = 1e-8,
) -> Float[Tensor, ""]:
    """Return the clipped surrogate objective, suitable for maximisation with gradient ascent.

    Note that for RLHF, we only care about the logprobs for the generated tokens, i.e. after the prefix. This is because
    we're fixing the prefix tokens and the model can't change its output for them, so there's no point including these
    in our objective function.

    logprobs:
        the logprobs of the action taken by the agent, according to the new policy
    mb_logprobs:
        logprobs of the actions taken in the sampled minibatch (according to the old policy)
    mb_advantages:
        advantages calculated from the sampled minibatch
    clip_coef:
        amount of clipping, denoted by epsilon in Eq 7.
    gen_len:
        the number of generated tokens, used for shape checking
    eps:
        used to add to std dev of mb_advantages when normalizing (to avoid dividing by zero)
    """
    assert (
        logprobs.shape[1] == mb_logprobs.shape[1] == mb_advantages.shape[1] == gen_len
    ), "Should pass in logprobs, mb_logprobs and mb_advantages for all generated tokens only, i.e. [:, -gen_len-1: -1]"

    logits_diff = logprobs - mb_logprobs

    r_theta = t.exp(logits_diff)

    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + eps)

    non_clipped = r_theta * mb_advantages
    clipped = t.clip(r_theta, 1 - clip_coef, 1 + clip_coef) * mb_advantages

    return t.minimum(non_clipped, clipped).mean()

In [18]:
def get_logprobs(
    logits: Float[Tensor, "batch seq_len vocab"],
    tokens: Int[Tensor, "batch seq_len"],
    prefix_len: int | None = None,
) -> Float[Tensor, "batch gen_len"]:
    """
    Returns correct logprobs for the given logits and tokens, for all the tokens after the prefix tokens (which have
    length equal to `prefix_len`).

    If prefix_len = None then we return shape (batch, seq_len-1).
    If not, then we return shape (batch, seq_len-prefix_len) representing the predictions for all toks after the prefix.
    """
    # Slice our tensors based on prefix_len
    if prefix_len is not None:
        logits = logits[:, prefix_len - 1 :]
        tokens = tokens[:, prefix_len - 1 :]

    # Get logprobs
    logprobs = logits.log_softmax(-1)

    # We want to get elements `logprobs[b, s, tokens[b, s+1]]`, we do this using eindex as follows:
    correct_logprobs = eindex(logprobs, tokens, "b s [b s+1]")

    return correct_logprobs


In [19]:


def get_optimizer(model: TransformerWithValueHead, base_lr: float, head_lr: float) -> t.optim.Optimizer:
    """
    Returns an AdamW optimizer for the model, with the correct learning rates for the base and head.
    """
    return t.optim.AdamW(
        [
            {"params": model.base_model.parameters(), "lr": base_lr},
            {"params": model.value_head.parameters(), "lr": head_lr},
        ],
        maximize=True,
    )


base_lr = 2e-5
head_lr = 5e-4
optimizer = get_optimizer(model, base_lr, head_lr)

assert len(optimizer.param_groups) == 2, "Your optimizer should have two parameter groups."
for param_group in optimizer.param_groups:
    assert param_group["maximize"], "Should be maximize=True."
    if len(param_group["params"]) <= 4:
        assert param_group["lr"] == head_lr, "LR for value head should be `head_lr`."
    else:
        assert param_group["lr"] == base_lr, "LR for base should be `base_lr`."

total_params = sum(len(param_group["params"]) for param_group in optimizer.param_groups)
assert total_params == len(
    list(model.parameters())
), "Your optimizer should have the same number of parameters as the model."

In [20]:

def get_optimizer_and_scheduler(args: RLHFArgs, model: TransformerWithValueHead):
    """
    Creates an AdamW optimizer and an LR scheduler that linearly warms up for `warmup_steps` steps, and then linearly
    decays to `final_scale` over the remaining steps.
    """

    def lr_lambda(step):
        assert step <= args.total_phases, f"Step = {step} should be less than total_phases = {args.total_phases}."
        if step < args.warmup_steps:
            return step / args.warmup_steps
        else:
            return 1 - (1 - args.final_scale) * (step - args.warmup_steps) / (args.total_phases - args.warmup_steps)

    optimizer = get_optimizer(model, args.base_lr, args.head_lr)
    scheduler = t.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    return optimizer, scheduler

In [21]:

class RLHFTrainer:
    model: TransformerWithValueHead
    ref_model: HookedTransformer
    memory: ReplayMemory  # we'll set this during rollout

    def __init__(self, args: RLHFArgs):
        t.manual_seed(args.seed)
        self.args = args
        self.run_name = f"{args.wandb_project_name}__seed{args.seed}__{time.strftime('%Y%m%d-%H%M%S')}"

        self.model = TransformerWithValueHead(args.base_model).to(device).train()
        self.ref_model = HookedTransformer.from_pretrained(args.base_model).to(device).eval()
        self.optimizer, self.scheduler = get_optimizer_and_scheduler(self.args, self.model)
        self.prefix_len = len(self.model.base_model.to_str_tokens(self.args.prefix, prepend_bos=self.args.prepend_bos))

    def compute_rlhf_objective(self, minibatch: ReplayMinibatch):
        """
        Computes the RLHF objective function to maximize, which equals the PPO objective function modified by the KL
        penalty term.

        Steps of this function are:
            - Get logits & values for the samples in minibatch
            - Get the logprobs of the minibatch actions taken
            - Use this data to compute all 4 terms of the RLHF objective function, and return it
            - Also optionally log stuff to Weights & Biases (and print some sample completions)
        """
        gen_len_slice = slice(-self.args.gen_len - 1, -1)  # define this for convenience

        # Get logits & values for our generated minibatch samples
        logits, values = self.model(minibatch.sample_ids)

        # Get logprobs for the the tokens generated (i.e. the logprobs of our actions)
        logprobs = get_logprobs(logits, minibatch.sample_ids, self.prefix_len)

        # Compute all terms of the loss function (including KL penalty)
        clipped_surrogate_objective = calc_clipped_surrogate_objective(
            logprobs, minibatch.logprobs, minibatch.advantages, self.args.clip_coef, self.args.gen_len
        )
        value_loss = calc_value_function_loss(
            values[:, gen_len_slice], minibatch.returns, self.args.vf_coef, self.args.gen_len
        )
        entropy_bonus = calc_entropy_bonus(logits[:, gen_len_slice], self.args.ent_coef, self.args.gen_len)
        kl_penalty = calc_kl_penalty(
            logits[:, gen_len_slice], minibatch.ref_logits[:, gen_len_slice], self.args.kl_coef, self.args.gen_len
        )

        # Compute net objective function
        ppo_objective_fn = clipped_surrogate_objective - value_loss + entropy_bonus
        total_objective_function = ppo_objective_fn - kl_penalty

        # Log stuff
        with t.inference_mode():
            logratio = logprobs - minibatch.logprobs
            ratio = logratio.exp()
            clipfracs = [((ratio - 1.0).abs() > self.args.clip_coef).float().mean().item()]
        if self.args.use_wandb:
            wandb.log(
                dict(
                    total_steps=self.step,
                    lr=self.scheduler.get_last_lr()[0],
                    clipped_surrogate_objective=clipped_surrogate_objective.item(),
                    clipfrac=np.mean(clipfracs),
                    value_loss=value_loss.item(),
                    values=values.mean().item(),
                    entropy_bonus=entropy_bonus.item(),
                    kl_penalty=kl_penalty.item(),
                ),
                step=self.step,
            )

        return total_objective_function

    def rollout_phase(self) -> ReplayMemory:
        """
        Performs a single rollout phase, retyrning a ReplayMemory object containing the data generated during this
        phase. Note that all forward passes here should be done in inference mode.

        Steps of this function are:
            - Generate samples from our model
            - Get logits of those generated samples (from model & reference model)
            - Get other data for memory (logprobs, normalized rewards, advantages)
            - Return this data in a ReplayMemory object
        """
        # Get our samples
        sample_ids, samples = get_samples(
            self.model.base_model,
            prompt=self.args.prefix,
            batch_size=self.args.batch_size,
            gen_len=self.args.gen_len,
            temperature=self.args.temperature,
            top_k=self.args.top_k,
            prepend_bos=self.args.prepend_bos,
        )

        # Generate logits from our model & reference model
        with t.inference_mode():
            logits, values = self.model(sample_ids)
            ref_logits = self.ref_model(sample_ids)

        # Get the logprobs of the generated tokens
        logprobs = get_logprobs(logits, sample_ids, self.prefix_len)

        # Calculate & normalize rewards (note we don't normalize inplace, because we want to log unnormalized rewards)
        rewards = self.args.reward_fn(samples)
        rewards_mean = rewards.mean().item()
        rewards_normed = normalize_reward(rewards) if self.args.normalize_reward else rewards

        # Compute advantages
        advantages = compute_advantages(values, rewards_normed, self.prefix_len)

        # Log stuff, and print output in a readable way (you could easily just regular print here instead of rprint table)
        if self.args.use_wandb:
            wandb.log({"mean_reward": rewards_mean}, step=self.step)

        n_log_samples = min(3, self.args.batch_size)
        ref_logprobs = get_logprobs(ref_logits[:n_log_samples], sample_ids[:n_log_samples], self.prefix_len).sum(-1)
        headers = ["Reward", "Ref logprobs", "Sample"]
        table_data = [[str(int(r)), f"{lp:.2f}", repr(s)] for r, lp, s in zip(rewards.tolist(), ref_logprobs, samples)]
        table = tabulate(table_data, headers, tablefmt="simple_grid", maxcolwidths=[None, None, 90])
        print(f"Phase {self.phase+1:03}/{self.args.total_phases:03}, Mean reward: {rewards_mean:.4f}\n{table}\n")

        return ReplayMemory(
            args=self.args,
            sample_ids=sample_ids,
            logprobs=logprobs,
            advantages=advantages,
            values=values,
            ref_logits=ref_logits,
        )

    def learning_phase(self, memory: ReplayMemory) -> None:
        """
        Performs a learning step on `self.memory`. This involves the standard gradient descent steps (i.e. zeroing
        gradient, computing objective function, doing backprop, stepping optimizer).

        You should also remember the following:
            - Clipping grad norm to the value given in `self.args.max_grad_norm`
            - Incrementing `self.step` by 1 for each minibatch
            - Stepping the scheduler (once per calling of this function)
        """
        for minibatch in memory.get_minibatches():
            self.optimizer.zero_grad()
            total_objective_function = self.compute_rlhf_objective(minibatch)
            total_objective_function.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=self.args.max_grad_norm)
            self.optimizer.step()
            self.step += 1

        self.scheduler.step()

    def train(self) -> None:
        """
        Performs a full training run.
        """
        self.step = 0
        self.samples = []

        if self.args.use_wandb:
            wandb.init(
                project=self.args.wandb_project_name,
                entity=self.args.wandb_entity,
                name=self.run_name,
                config=self.args,
            )

        for self.phase in range(self.args.total_phases):
            memory = self.rollout_phase()
            self.learning_phase(memory)

        if self.args.use_wandb:
            wandb.finish()

In [22]:
# Testing your setup: kl_coef=0.0 (see dropdown above the previous code block for explanation)
args = RLHFArgs(use_wandb=False, kl_coef=0.0, total_phases=30, warmup_steps=0, reward_fn=reward_fn_char_count)
trainer = RLHFTrainer(args)
trainer.train()

Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  mps
Phase 001/030, Mean reward: 1.4844
┌──────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────┐
│   Reward │   Ref logprobs │ Sample                                                                                 │
├──────────┼────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│        1 │         -69.77 │ '<|endoftext|>This is a list of the best games in Overwatch and its community.\n\nThis │
│          │                │ list is compiled from the most popular and popular game of each year, which'           │
├──────────┼────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│        3 │         -11.08 │ '<|endoftext|>This is a rush transcript. Copy may not be in its final form.\

In [23]:
args = RLHFArgs(use_wandb=True, reward_fn=reward_fn_char_count)  
trainer = RLHFTrainer(args)
trainer.train()
     

Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  mps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: michaelyliu6 (michaelyliu6-none). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoi

Phase 001/100, Mean reward: 1.4844
┌──────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────┐
│   Reward │   Ref logprobs │ Sample                                                                                 │
├──────────┼────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│        1 │         -69.77 │ '<|endoftext|>This is a list of the best games in Overwatch and its community.\n\nThis │
│          │                │ list is compiled from the most popular and popular game of each year, which'           │
├──────────┼────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│        3 │         -11.08 │ '<|endoftext|>This is a rush transcript. Copy may not be in its final form.\n\nAMY     │
│          │                │ GOODMAN: That is Democracy Now!, democracynow.org, The War'                            │
├──────────┼─

clipfrac,▁▂▁▃▃▁▂▃▁█▁▁▇▇▁▁▆▁▇▇▂▄▅▁▂▃▅▂▂▂▃▁▃▁▃▁▁▂▂▁
clipped_surrogate_objective,▄▂▂▄▂██▂▂▂▆▂▇▃▆▆▅▅▄▂▂▂▃▆▃▂▅▃▁▂▂▄▇▂▂▂▅▅▃▂
entropy_bonus,▆▆▆▄▅▆▇▆▆▇▇▆▅██▅▅▄▅▃▅▅▂▁▄▃▄▅▄▆▄▄▃▅▂▂▂▂▂▂
kl_penalty,▁▂▂▂▂▂▃▃▄▄▅▆▅▄▅▅▅▆▆▆▇▆▇▆▆▆▆▇▇▇▆█▇█▇█▇▇▇▇
lr,▁▂▃▅▅▇██▇▇▇▇▇▇▇▇▇▇▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂
mean_reward,▃▃▂▃▂▂▄▂▁▂▂▃▅▄▂▇▅▅▅▆▇▇▆█▇█▆▄▄▇▆▆▇▇█▇████
total_steps,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███
value_loss,▄▄▄▆▄▇██▃▂▃▁▃▃▃▂▃▂▂▁▂▃▂▂▂▂▃▂▂▃▁▂▄▁▃▂▂▁▂▃
values,▁▁▂▂▂▂▃▂▄▄▄▅▄▅▆▆▇▆▆▇█▆▅▅▅▆█▆██▆▇█▆▆▇█▇█▆
clipfrac,0.00521
clipped_surrogate_objective,0.01113
